<a href="https://colab.research.google.com/github/renatovfreitas/DataCircle_Twitter_Project/blob/task/twitterSentimentLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import numpy as np
import pandas as pd
import nltk
from textblob import TextBlob
from nltk.corpus import twitter_samples, stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
import re # Import the re module for regular expressions
import string
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer  # Import TweetTokenizer
from nltk.stem import PorterStemmer  # Import PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score


In [92]:
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Pre-processing of tweets include the following steps:

1.   Removing punctuations, hyperlinks and hashtags
2.   Tokenization — Converting a sentence into list of words
3. Converting words to lower cases
4. Removing stop words
5. Lemmatization/stemming — Transforming a word to its root word



In [93]:
from google.colab import files
uploaded = files.upload()

Saving thousand_tweets.csv to thousand_tweets (2).csv


In [94]:
df = pd.read_csv("thousand_tweets.csv")

In [95]:
print(df.head(5))

            created_at             tweet_id  likes  retweet_count  \
0  2020-10-15 07:15:00  1316638690953891840      0              0   
1  2020-11-07 14:20:18  1325080641336193024      1              0   
2  2020-10-16 12:42:31  1317083500428759040      2              1   
3  2020-11-07 16:39:47  1325115746138124288     17              0   
4  2020-10-25 14:11:58  1320367504141660160    238             61   

                source    user_id       user_join_date  user_followers_count  \
0      twitter web app  178730312  2010-08-15 14:56:43                   642   
1   twitter for iphone   37132337  2009-05-02 03:44:26                   231   
2  twitter for android  371229186  2011-09-10 12:59:32                  1293   
3     twitter for ipad  407826450  2011-11-08 16:15:48                   631   
4      twitter web app   63227112  2009-08-05 19:21:04                 31871   

              user_location           city         country       state  \
0    wherever i may roam...   

Preprocess the Tweets: Use the process_tweet function to clean and tokenize the tweets in DataFrame.

# Check for NaN values and convert to string

In [97]:
print(df.columns)

Index(['created_at', 'tweet_id', 'likes', 'retweet_count', 'source', 'user_id',
       'user_join_date', 'user_followers_count', 'user_location', 'city',
       'country', 'state', 'candidate', 'tweet_cleaned'],
      dtype='object')


# Define the process_tweet function

In [98]:
def process_tweet(tweet):
    tweet = re.sub(r'\$\w*', '', tweet)  # Remove dollar signs and words following them
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)  # Remove URLs
    tweet = re.sub(r'#', '', tweet)  # Remove hash symbol
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    stopwords_english = stopwords.words('english')
    stemmer = PorterStemmer()
    tweets_stem = []

    for word in tweet_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            stem_word = stemmer.stem(word)
            tweets_stem.append(stem_word)

    return tweets_stem

# Process the tweets in your DataFrame

In [99]:
df['processed_tweets'] = df['tweet_cleaned'].apply(process_tweet)
print(df[['tweet_cleaned', 'processed_tweets']].head(5))

                                       tweet_cleaned  \
0  trade deal #trump could open #nhs us corporati...   
1  lilduval #biden america act like theyre scared...   
2  wouldnt denounce #qanon poisonous conspiracy t...   
3  2020 might redeemed #americadecides2020 ##bide...   
4  trump amp co gave chief staff said explicitly ...   

                                    processed_tweets  
0  [trade, deal, trump, could, open, nh, us, corp...  
1  [lilduv, biden, america, act, like, theyr, sca...  
2  [wouldnt, denounc, qanon, poison, conspiraci, ...  
3  [2020, might, redeem, americadecid, 2020, bide...  
4  [trump, amp, co, gave, chief, staff, said, exp...  


In [100]:
# Sentiment polarity function
def get_polarity(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    return polarity

# Apply sentiment polarity function to the DataFrame
df['polarity'] = df['tweet_cleaned'].apply(get_polarity)

In [101]:
# Subjectivity function
def get_subjectivity(text):
    analysis = TextBlob(text)
    if analysis.sentiment.subjectivity < 0.5:
        subjectivity = 'objective'
    else:
        subjectivity = 'subjective'

    return subjectivity

# Apply subjectivity function to the DataFrame
df['subjectivity'] = df['tweet_cleaned'].apply(get_subjectivity)

In [102]:
# Create a column for Sentiment analysis
df['sentiment'] = df['polarity'].apply(lambda i: 'positive' if i > 0 else ('neutral' if i == 0 else 'negative'))

# Create a set of unique words


In [103]:
all_words = []
for tweet in df ['processed_tweets']:
  all_words.extend(tweet)
word_l = list(set(all_words))

# Create feature matrix X and labels y
# Initialize the feature matrix


In [104]:
X = np.zeros((len(df), len(word_l) +1)) # +1 for bias term
# Create the labels based on the sentiment column
df['sentiment_label'] = df['sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 0.5})
y = df['sentiment_label'].values.reshape(-1, 1)  # Reshape y to be (m, 1)

In [105]:
# Fill the feature matrix
for i, tweet in enumerate(df['processed_tweets']):
    for word in tweet:
        if word in word_l:
            index = word_l.index(word)
            X[i, index + 1] += 1  # +1 for the bias term

# Add bias term (column of ones)
X[:, 0] = 1  # Bias term


In [106]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize theta
theta = np.zeros((X_train.shape[1], 1))  # Initialize theta with zeros

# Define the sigmoid and gradient descent functions


In [107]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

def gradientDescent(X, y, theta, alpha, num_iters):
    m = len(y)  # Number of training examples
    J_history = []  # To store the cost at each iteration

    for i in range(num_iters):
        z = np.dot(X, theta)  # Linear combination
        h = sigmoid(z)  # Hypothesis
        J = (-1/m) * (np.dot(y.T, np.log(h)) + np.dot((1 - y).T, np.log(1 - h + 1e-15)))  # Cost function
        grad = (1/m) * np.dot(X.T, (h - y))  # Gradient
        theta = theta - alpha * grad  # Update theta
        J_history.append(J[0, 0])  # Store the cost

    return theta, J_history

In [108]:
print("x shape:", X.shape)  # Should be (m, n)
print("y shape:", y.shape)  # Should be (m,) or (m, 1)
print("theta shape:", theta.shape)  # Should be (n,) or (n, 1)

x shape: (1000, 4582)
y shape: (1000, 1)
theta shape: (4582, 1)


# Prepare to train the model


In [109]:
# Train the logistic regression model
alpha = 0.01
num_iters = 1000
theta, J_history = gradientDescent(X_train, y_train, theta, alpha, num_iters)

In [110]:
# Predict sentiment for the test set
y_pred = sigmoid(np.dot(X_test, theta))
y_pred_labels = np.zeros(y_pred.shape)
for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred_labels[i] = 1
    else:
        y_pred_labels[i] = 0
y_pred_prob = sigmoid(np.dot(X_test, theta))  # Get probabilities
y_pred_labels = (y_pred_prob >= 0.5).astype(int)  # Convert probabilities to binary labels

In [111]:
y_pred_labels = (y_pred > 0.5).astype(int)  # Adjust the threshold if necessary

# Convert y_test to binary labels (1 for positive, 0 for negative)
y_test_binary = np.where(y_test == 0.5, 1, y_test).astype(int).flatten()

# Ensure both y_test and y_pred_labels are of the same shape and type
print("Shape of y_test:", y_test.shape)
print("Shape of y_pred_labels:", y_pred_labels.shape)

Shape of y_test: (200, 1)
Shape of y_pred_labels: (200, 1)


In [112]:
# Evaluate the model
accuracy = accuracy_score(y_test_binary, y_pred_labels.flatten())
confusion = confusion_matrix(y_test_binary, y_pred_labels.flatten())

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")

Accuracy: 0.84
Confusion Matrix:
[[  4  26]
 [  6 164]]
